# Bias-variance tradeoff

Let us analyze the squared error of trained models. We will assume the existence of a a ground truth function $f$. Let $f_{\mathcal{D}}$ be the function obtained by the training process over the sample $\mathcal{D}.$ Let $\bar{f}$ be the expected function when drawing fixed sized samples $\mathcal{D} \stackrel{\text{iid}}{\sim} P^N$ where $N = |\mathcal{D}|$ and $P$ is the underlying data distribution. In other words, $\bar{f}$ is an ensemble of trained models weighted by the probability of its training dataset. Then:

$$
\begin{aligned}
\mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[ \left(f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}) \right)^2 \right]
&= \mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[ \left((f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - \bar{f}(\boldsymbol{\mathsf{x}})) + (\bar{f}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}})) \right)^2 \right] \\
&= \mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[ (f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - \bar{f}(\boldsymbol{\mathsf{x}}))^2 \right] 
+ 2\, \mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[(f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}))(\bar{f}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}})) \right] + \mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[(\bar{f}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}))^2 \right] \\
&= \mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[ (f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - \bar{f}(\boldsymbol{\mathsf{x}}))^2 \right] 
+ 0 + \mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[(\bar{f}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}))^2 \right] \\
&= \underbrace{\mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[ (f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - \bar{f}(\boldsymbol{\mathsf{x}}))^2 \right]}_{\text{Variance}} 
 + \underbrace{\mathbb{E}_{\boldsymbol{\mathsf{x}}}\left[(\bar{f}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}))^2 \right]}_{\text{Bias}^2}. \\
\end{aligned} \\
$$

The middle term vanishes by writing it as: 

$$
\mathbb{E}_{\boldsymbol{\mathsf{x}}} \left[\underbrace{\mathbb{E}_{\mathcal{D}}\left[(f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}))\right]}_{0} \; (\bar{f}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}))  \right] = 0.
$$

Observe that the **variance** term describes variability of models as we re-run the training process without actually looking into the true function. The **bias** term, on the other hand, looks at the error of the ensemble $\bar{f}$ from the true function $f.$ These can be visualized as manifesting in the left and right plots respectively of {numref}`01-overfitting-underfitting`.

**Remark.** This derivation ignores target noise which is relevant in real-world datasets. Here we have a distribution $p(y \mid \boldsymbol{\mathsf{x}})$ around the target which we have to integrate over to get the expected target. See [these notes](https://www.cs.cornell.edu/courses/cs4780/2018sp/lectures/lecturenote12.html) for a more careful treatment.

<br>

## Classical tradeoff

There is a tradeoff with respect to model complexity for fixed $n.$ If we increase model complexity, these models will fit each sample too well that it captures even sampling noise. However, these cancels out over many samples resulting in low bias. Overfitting occurs because we get a model that performs well on the training sample, but may not perform well on a test sample due to high variability. On the other hand, simpler models tend to have high bias and underfits any sample of the dataset. Note that using a sufficiently large sample size for a fixed model class can decrease variance as more data will result in drowning out noise. But bias does not go away no matter how much data we have! {cite}`barron` provides an explicit tradeoff between data and network size for a certain class of fully-connected neural networks and training algorithm:

$$
\mathbb{E}_{\boldsymbol{\mathsf{x}}, \mathcal{D}}\left[ \left(f_{\mathcal{D}}(\boldsymbol{\mathsf{x}}) - f(\boldsymbol{\mathsf{x}}) \right)^2 \right] \leq O\left(\frac{1}{M}\right) + O\left(\frac{Md}{N}\right)\log N
$$

where $M$ is the number of nodes, $N = |\mathcal{D}|$ is the number of training observations, and $d$ is the input dimension. Here the first term corresponds to the bias which is data independent, while the second term corresponds to the variance which increases with network size and decreases with data. This bound also highlights the **curse of dimensionality**. The input dimension contributes linearly to the error, while data only decreases error at the rate $O(\log N / N)$.

<br>

## Double descent

Moreover, there is the phenomenon of **double descent** {cite}`double-descent` observed in most networks used in practice where both bias and variance go down with excess complexity. See [this interview](https://www.youtube.com/watch?v=W_TAKJRgrbs) by one of the authors. One intuition is that near the **interpolation threshold** where there is roughly a 1-1 correspondence between all the sample datasets and the models, small changes in the dataset lead to large changes in the model. This is the **critical regime** in the classical case where overfitting occurs. Having more data destroys this 1-1 correspondence, which is covered by the classical tradeoff described above. 

Double descent occurs in the opposite case where we have much more parameters than data ({numref}`01-double-descent`). SGD gets to focus more on what it wants to do, i.e. search for **flat minima** {cite}`flat-minima`, since it is not constrained to use the full model capacity. Note that the double descent curve is more prominent when there is label noise. In this case, the complexity tradeoff is sharper in the critical regime since there is less redundancy in parameters of the model when the dataset is harder to learn.

**Remark.** Models around with weights flat minimas have validation errors that are much more stable to perturbation in the weights and, as such, tend to be smooth between data points {cite}`hochreiter1997flat`. SGD is discussed in the next chapter.

<br>

```{figure} ../../img/nn/01-double-descent.png
---
name: 01-double-descent
width: 80%
---
Double descent for ResNet18. The width parameter controls model complexity. Source: {cite}`double-descent`
```

<br>

```{figure} ../../img/nn/01-double-descent-data.png
---
name: 01-double-descent-data
width: 90%
---
Additional data increases model variance within the critical regime.
More data shifts the interpolation threshold to the right, resulting in
worse test performance compared to the same model trained on a smaller sample. 
Increasing model complexity improves test performance.
Source: {cite}`double-descent`
```

<br>

```{figure} ../../img/nn/01-double-descent-epochs.png
---
name: 01-double-descent-epochs
width: 100%
---
Epoch dimension to double descent. Models are ResNet18s on CIFAR10
with 20% label noise, trained using Adam with learning rate 0.0001, and data augmentation. 
**Left:** Training dynamics for models in three regimes. **Right:** Test error vs. Model size × Epochs. 
Three slices of this plot are shown on the left. Source: {cite}`double-descent`
```